# Sea Ice example notebook for timeseries

---

This notebook shows how to use the seaice diagnostic. It contains an example of access of a reference (OSI-SAF or PSC) and model (IFS-NEMO) datasets to evaluate 
and plot the sea ice extension, volume, fraction and thickness variables.

The only import needed is the `SeaIce` and `PlotSeaIce` and `Plot2DSeaIce` classes from the `aqua.diagnostics` module.

In [ ]:
from aqua.diagnostics import SeaIce, PlotSeaIce, Plot2DSeaIce

- The `SeaIce` class allow to retrive, regrid and perform the main analysis on the data. It also allow to save the resulting data as NetCDF file (through the method `save_netcdf`). 
- The `PlotSeaIce` class allow to plot and evaluate the `extent` and `volume` of sea ice.
- The `Plot2DSeaIce` class allow to plot 2D maps and evaluate the differences of `fraction` and `thickness` of sea ice.

## Data retrieve and sea ice calculation

At first it must be initialised the class `SeaIce`, which calls internally the `Reader` and performs the regridding (if a `regrid` option is provided).  
Then we compute the sea ice for the model data (using the method `compute_seaice`).  
The `regions` argument allow to choose the specific regions defined by lat-lon in `regions.yaml` file. 

## Seaice extent

### Calculate: model data - surface extent 

In [ ]:
method   = 'extent'   # This is the method we want to compute with the seaice data
varname  = 'siconc'   # This is the right variable name relative to the desired method for IFS-NEMO
source   = 'lra-r100-monthly' # source name can change depending on the model and exp
regions_domain = ['arctic','antarctic']
regrid='r100'
loglevel = 'info'

seaice_model = SeaIce(model='IFS-NEMO', exp='historical-1990', source=source, regions=regions_domain, 
                      startdate='1991-01-01', enddate='2000-01-01', 
                      regrid=regrid, loglevel='info')
simod = seaice_model.compute_seaice(method=method, var=varname)

- The data can be saved into NetCDF files:

In [ ]:
seaice_model.save_netcdf(simod, 'seaice', diagnostic_product='timeseries', extra_keys={'method': method, 'source': source, 'regions_domain': "_".join(regions_domain)})

### Calculate: reference sea ice - surface extent 

The same must be done for the observations (reference data).  
Since we want to plot both the 'arctic' and 'antarctic' region, we need to load both the data Northern 
and Southern Hemisphere separately due to how OSI-SAF provide these datasets.  
Then to compute the sea ice extent. We also want to calculate the standard deviation for the observations for an improved comparison of the model against onservations.  
Since only the plotting of the monthly timeseries is supported at the moment for the sea ice, we must set `calc_std_freq = 'monthly'`. This will create also datasets 
with the std calcualted from the data.

In [ ]:
seaice_ref_nh = SeaIce(model='OSI-SAF', exp='osi-450', source='nh-monthly', regions='arctic',
                       startdate=startdate, enddate=enddate, 
                       regrid=regrid, loglevel=loglevel)
si_ref_nh, si_std_ref_nh = seaice_ref_nh.compute_seaice(method=method, var=varname, calc_std_freq='monthly')

seaice_ref_sh = SeaIce(model='OSI-SAF', exp='osi-450', source='sh-monthly', regions='antarctic',
                      startdate=startdate, enddate=enddate, 
                      regrid=regrid, loglevel=loglevel)
si_ref_sh, si_std_ref_sh = seaice_ref_sh.compute_seaice(method=method, var=varname, calc_std_freq='monthly')

Following example above, the data can be saved in NetCDF.

### Plot: reference and model - surface extent 

To plot the seaice timeseries, we first instantiate the `PlotSeaIce` object and then plot it with the `plot_seaice` method.

- The `plot_seaice` method accepts a single xr.DataArray, a xr.Dataset or a list of xr.Dataset.  
Switch *save_pdf* *_png* to *True* to save the figure in the selected *outputdir*:

In [ ]:
psi = PlotSeaIce(monthly_models=simod,
                 monthly_ref=[si_ref_nh, si_ref_sh],
                 monthly_std_ref=[si_std_ref_nh, si_std_ref_sh], 
                 outputdir='.',
                 rebuild=True,
                 loglevel=loglevel)
psi.plot_seaice(plot_type='timeseries', save_pdf=False, save_png=False)

---

## Seaice volume 

Similar steps as above must be followed.

### Calculate: model data - volume

In [ ]:
method   = 'volume'    # This is the method we want to compute with the seaice data
varname  = 'sithick'   # This is the right variable name relative to the desired method for IFS-NEMO
source   = 'lra-r100-monthly'
regions_domain = ['arctic','antarctic']
regrid='r100'
loglevel = 'info'

seaice_model = SeaIce(model='IFS-NEMO', exp='historical-1990', source=source, regions=regions_domain,
                      startdate='1991-01-01', enddate='2000-01-01',
                      regrid=regrid, loglevel='info')
simod = seaice_model.compute_seaice(method=method, var=varname)

### Calculate: reference sea ice - volume 

In [ ]:
varname = 'sivol'

seaice_ref_nh = SeaIce(model='PSC', exp='PIOMAS', source='monthly', regions='arctic',
                       startdate='1991-01-01', enddate='2000-01-01',
                       regrid=regrid, loglevel=loglevel)
si_ref_nh, si_std_ref_nh = seaice_ref_nh.compute_seaice(method=method, var=varname, calc_std_freq='monthly')

seaice_ref_sh = SeaIce(model='PSC', exp='GIOMAS', source='monthly', regions='antarctic',
                       startdate='1991-01-01', enddate='2000-01-01', 
                       regrid=regrid, loglevel=loglevel)
si_ref_sh, si_std_ref_sh = seaice_ref_sh.compute_seaice(method=method, var=varname, calc_std_freq='monthly')

### Plot: reference and model - volume 

To plot the seaice timeseries, we first instantiate the `PlotSeaIce` object and then plot it with the `plot_seaice` method.

- The `plot_seaice` method accepts a single xr.DataArray, a xr.Dataset or a list of xr.Dataset.  
Switch *save_pdf* *_png* to *True* to save the figure in the selected *outputdir*:

In [ ]:
psi = PlotSeaIce(monthly_models=simod,
                 monthly_ref=[si_ref_nh, si_ref_sh],
                 monthly_std_ref=[si_std_ref_nh, si_std_ref_sh], 
                 outputdir='.',
                 rebuild=True,
                 loglevel=loglevel)
psi.plot_seaice(plot_type='timeseries', save_pdf=False, save_png=False)